<a href="https://colab.research.google.com/github/h0806449f/PyTorch/blob/main/DB_05_to_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **=== 0. 簡介 ===**
1. Run from command line
2. Final target structure


## 0.1 Example for run python from command line




In [1]:
# python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS

In [2]:
# python train.py --model model_1 --batch_size 100 --lr 0.001 --num_epochs 50

## 0.2 Final target structure

In [3]:
# going_modular/
# ├── going_modular/
# │   ├── data_setup.py
# │   ├── engine.py
# │   ├── model_builder.py
# │   ├── train.py
# │   └── utils.py
# ├── models/
# │   ├── 05_going_modular_cell_mode_tinyvgg_model.pth
# │   └── 05_going_modular_script_mode_tinyvgg_model.pth
# └── data/
#     └── pizza_steak_sushi/
#         ├── train/
#         │   ├── pizza/
#         │   │   ├── image01.jpeg
#         │   │   └── ...
#         │   ├── steak/
#         │   └── sushi/
#         └── test/
#             ├── pizza/
#             ├── steak/
#             └── sushi/

# **=== 1. Python files ===**
1. Get data (download data)
2. Data_setup.py

## 1.1 Get data (download data)
Download the data and set into specific data dtructure

In [4]:
# Import
import requests             # HTTP request
import zipfile              # zip & unzip
import os                   #
from pathlib import Path    #

# 設置 data path 變數
data_folder = Path("Data/")
image_folder = data_folder / "Pizze_Steak_Sushi"

# 判斷是否已有資料夾 -> 無: 創造 / 有: 可重複接受
if image_folder.is_dir():
    print(f"{image_folder} 資料夾已存在")
else:
    print(f"{image_folder} 未發現資料夾, 創造 ...")
    image_folder.mkdir(parents = True, exist_ok = True) # 無上層資料夾, 也會創造 image_folder 資料夾 / 已有資料夾, 不返回error

# 下載
with open(image_folder / "pizza_steak_sushi.zip", "wb") as f: # 名稱需一致 (pizza_steak_sushi.zip)
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("downloading ...")
    f.write(request.content)

# 解壓縮
with zipfile.ZipFile (image_folder / "pizza_steak_sushi.zip", "r") as zip_f:
    print("unziping ...")
    zip_f.extractall(image_folder)

# 移除 壓縮檔
os.remove(image_folder / "pizza_steak_sushi.zip")

Data/Pizze_Steak_Sushi 未發現資料夾, 創造 ...
downloading ...
unziping ...


## 1.2 Data_setup.py
1. Wrtie -> functions in **Data_setup.py**
2. Test
3. Import

### 1.2.1 Wrtie -> functions in **Data_setup.py**

In [28]:
# writefile 將以下內容, 寫入指定檔案中 (須去除前置# writefile 才會正常運作)
#%%writefile going_modular/Data_setup.py
"""將創造 Pytorch DataLoader 用於影像分類"""

# Import
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Create function - create_dataloader()
# 預先設置參數的資料型態: 日後易維護, IDE可能偵錯

NUM_WORKERS = os.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=None):
    # 解說
    """
    創造:
        train_dataloader, test_dataloader

    變數:
        train_dir: 訓練資料路徑
        test_dir: 測試資料路徑
        transform: Pytorch torchvision transforms
        batch_size: 每批次資料數量
        num_workers: workers per DataLoader

    返還:
        tuple(train_dataloader, test_dataloader, class_names)
        class_names 為list 包含訓練資料的 classes

    範例:
        train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=path/to/train_dir,
                                                                            test_dir=path/to/test_dir,
                                                                            transform=some_transform,
                                                                            batch_size=32,
                                                                            num_workers=4)
    """

    # NUM_WORKERS
    if num_workers is None:
        num_workers = NUM_WORKERS

    # ImageFolder -> datasset
    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    test_dataset = datasets.ImageFolder(test_dir, transform=transform)

    # Get train_dataset class
    class_name = train_dataset.classes

    # Dataset -> dataloader
    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=True) # pin_memory 使用GPU時 可加快處理速度

    test_dataloader = DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=num_workers,
                                 pin_memory=True)

    return train_dataloader, test_dataloader, class_name

### 1.2.1 Test

In [34]:
import torch
from torchvision import transforms
from torchvision.transforms import ToTensor

train_path = "/content/Data/Pizze_Steak_Sushi/train"
test_path = "/content/Data/Pizze_Steak_Sushi/test"

transform = transforms.Compose([
    ToTensor()
])

create_dataloader(train_dir=train_path,
                  test_dir=test_path,
                  transform=transform,
                  batch_size=100)

(<torch.utils.data.dataloader.DataLoader at 0x7f1708dc70d0>,
 ['pizza', 'steak', 'sushi'])

### 1.2.2 Import

In [35]:
from going_modular import Data_setup

create_dataloader(train_dir=train_path,
                  test_dir=test_path,
                  transform=transform,
                  batch_size=100)

(<torch.utils.data.dataloader.DataLoader at 0x7f1641f75900>,
 ['pizza', 'steak', 'sushi'])

## 1.3 Model.py
1. Write -> functions in Model.py
2. Test
3. Import

### 1.3.1 Write -> functions in Model.py

In [38]:
# writefile 將以下內容, 寫入指定檔案中 (須去除前置# writefile 才會正常運作)
# %%writefile going_modular/Model.py
"""將創造 tinyVGG 模型"""

# Import
import torch
from torch import nn

### 1.3.2 Test

### 1.3.3 Import